<a href="https://colab.research.google.com/github/MatheusAlvesCavalcante/ufcinova-scraper/blob/main/web_scraper_UFC_Inova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Raspagem inicial**

In [ ]:
# Importações necessárias
import time, requests, json, csv, pprint
import pandas as pd
from bs4 import BeautifulSoup as bp
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor, as_completed


'''Raspagem da página inicial'''

# URL da página principal
url_pag_inicial = "https://ufcinova.ufc.br/pt/vitrinetecnologica/"

# Identificador do navegador - Ajuda a evitar bloqueios do site
cabecalho = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:118.0) Gecko/20100101 Firefox/118.0"
}

# Conjunto global para controlar links já visitados
vistos = set()

session = requests.Session()
session.headers.update(cabecalho)


# Função que retorna um JSON das páginas encontradas na primeira página
def analisar_primeira(url):
    """
    Raspa a pagina inicial para encontrar os links das categorias de tecnologias.
    """
    print(f"Buscando categorias em {url}")
    pagina = session.get(url, timeout=20)
    pagina.raise_for_status()

    parser = bp(pagina.text, "html.parser")

    catalogo = []

    # Seleciona os links dentro de títulos de tecnologias
    for link in parser.select("h5.fg-item-title a[href], .fg-item-title a[href]"):
        titulo = link.get_text(strip=True)
        url_encontrada = urljoin(url, link.get("href"))

        # Evita duplicação de URLs
        if titulo and url_encontrada and url_encontrada not in vistos:
            catalogo.append({
                "titulo": titulo,
                "url": url_encontrada,
                "lista_origem": url
            })
            vistos.add(url_encontrada)

    print(f"Categorias encontradas: {len(catalogo)}")
    return catalogo



# 1. Importações (As Ferramentas)
* import requests: Usado para fazer o download da página da web (o código HTML).


* import json: Usado para formatar a lista de dados final em um texto JSON
legível.

* import pandas as pd: Importado no seu código, mas não está sendo usado neste trecho. (Geralmente é usado para organizar dados em tabelas).

* from bs4 import BeautifulSoup: A ferramenta principal. Ela "lê" o HTML que o requests baixou e permite encontrar partes específicas dele (como links ou títulos).

* from urllib.parse import urljoin: Uma função pequena para montar URLs completas. Se ele acha um link como /pt/tecnologia/123, ele o junta com a URL base (https://ufcinova.ufc.br) para formar o link completo.

# 2. Configurações Iniciais (A Preparação)
* url_pag_inicial = "...": Define o "alvo" — qual página você quer raspar.

* cabecalho = {"User-Agent": "..."}: É um "disfarce". Faz seu script se identificar para o site como se fosse um navegador Firefox comum, o que ajuda a evitar que o site bloqueie o acesso.

* vistos = set(): Cria uma "lista de memória" (tecnicamente um "conjunto") vazia. Ela será usada para guardar todos os links já encontrados, garantindo que o script não colete o mesmo link duas vezes.

# 3. A Função analisar_primeira(url)
* Isso define um bloco de código que faz o trabalho de raspagem:

* pagina = requests.get(...): Baixa o conteúdo da URL, usando o "disfarce" (cabecalho).

* pagina.raise_for_status(): Verifica se o download deu certo. Se o site der um erro (como "Página não encontrada"), o script para.

* parser = BeautifulSoup(...): Entrega o HTML baixado para o BeautifulSoup organizar e preparar para a busca.

* catalogo = []: Cria uma lista vazia onde os dados encontrados (título e url) serão guardados.

* for link in parser.select(...): Este é o comando de busca. Ele diz ao BeautifulSoup: "Encontre todos os links (a[href]) que estejam dentro de um elemento HTML com a classe fg-item-title".

* titulo = link.get_text(strip=True): Para cada link encontrado, pega o texto visível dele (o título) e remove espaços em branco inúteis do início e do fim.

* url_encontrada = urljoin(...): Pega o endereço do link e (usando o urljoin) o transforma em uma URL completa.

* if ... and url_encontrada not in vistos:: Faz uma checagem: o título não está vazio? A URL não está vazia? E, o mais importante, essa URL já não está na nossa "lista de memória" (vistos)?

* catalogo.append(...): Se a checagem passar, ele adiciona os dados (título, url, url de origem) à lista catalogo.

* vistos.add(url_encontrada): Adiciona a URL na "lista de memória" (vistos) para não ser pega de novo.

* return catalogo: Quando o for termina, a função devolve a lista catalogo cheia de dados.

# **Raspagem da segunda página**

In [ ]:
def analisar_subpaginas(lista_categorias):
    """itera sobre cada página de categoria com paginação, extraindo links de tecnologias em
    conjunto
    """
    todas_tecnologias = []
    urls_vistas_tech = set()

    for categoria in lista_categorias:
        url_categoria = categoria['url']
        nome_categoria = " ".join(categoria['titulo'].split())
        print(f"\nProcessando categoria: {nome_categoria}")

        pagina_atual = 1
        while True:
            if pagina_atual == 1:
                url_pagina = url_categoria
            else:
                url_pagina = url_categoria.rstrip('/') + f"/page/{pagina_atual}/"

            print(f"Acessando página: {url_pagina}")

            pagina = session.get(url_pagina, timeout=20)
            if pagina.status_code == 404:
                break

            parser = bp(pagina.text, "html.parser")
            links = parser.select("h2 a[href], h5 a[href], .fg-item-title a[href]")

            if not links:
                break
            novos_links_encontrados = 0
            for link in links:
                titulo = link.get_text(strip=True)
                url_tec = urljoin(url_pagina, link.get("href"))

                if url_tec not in urls_vistas_tech and url_tec not in vistos:
                    todas_tecnologias.append({
                        "titulo": titulo,
                        "url": url_tec,
                        "categoria": nome_categoria
                    })
                    urls_vistas_tech.add(url_tec)
                    novos_links_encontrados += 1

            if novos_links_encontrados == 0:
                break

            pagina_atual += 1
            time.sleep(0.3)

    print(f"\nTotal de tecnologias únicas capturadas: {len(todas_tecnologias)}")
    return todas_tecnologias

In [ ]:
def extrair_info_tecnologia(url):
    """
    Acessa a URL de uma tecnologia e extrai detalhes.
    """
    resposta = session.get(url, timeout=20)
    resposta.raise_for_status()

    soup = bp(resposta.text, 'html.parser')

    #TÍTULO
    titulo_tag = soup.find("title")
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else "Sem título"

    #DESCRIÇÃO
    bloco_descricao = soup.select("div.elementor-widget-text-editor p")
    descricao = "\n".join(filter(None, [p.get_text(strip=True) for p in bloco_descricao]))

    #BENEFÍCIOS
    beneficios_lista = []
    blocos_beneficios = soup.select("h4.elementor-icon-box-title span")
    for span in blocos_beneficios:
        texto = span.get_text(strip=True)
        if texto and "status" not in texto.lower() and "trl" not in texto.lower():
            beneficios_lista.append(texto)
    beneficios = "\n".join(beneficios_lista)

    # STATUS
    status = "Não informado"
    for el in blocos_beneficios:
        texto = el.get_text(strip=True)
        if "status:" in texto.lower():
            status = texto.replace("Status:", "").strip()
            break

    # TRL
    trl = "Não informado"
    trl_elements = soup.select("div.elementor-icon-box-description p, div.elementor-icon-box-content span, div.elementor-icon-box-content h4")
    for el in trl_elements:
        text = el.get_text(strip=True)
        if text.startswith("TRL"):
            trl = text
            break

    # INVENTORES
    inventores_set = set()
    inventores_tags = soup.select("p.eael-team-text")
    for p in inventores_tags:
        texto = p.get_text(strip=True)
        if texto and "@" not in texto and all(x not in texto for x in ["Fone", "Departamento", "Contato"]):
            inventores_set.add(texto)

    if not inventores_set:
        name_tags = soup.select(".eael-team-member-name")
        inventores_set.update(tag.get_text(strip=True) for tag in name_tags)

    # DEPARTAMENTO
    departamento = ""
    depto_tag = soup.find(string=lambda text: text and "Departamento de" in text)
    if depto_tag:
        departamento = depto_tag.strip()

    # CONTATO
    partes = []
    for p in soup.find_all("p"):
        txt = p.get_text(strip=True)
        if any(x in txt for x in ["Fone:", "E-mail:", "Contato"]):
            partes.append(txt)
    contato = "\n".join(partes)

    return {
        "Título": titulo,
        "Descrição": descricao,
        "Benefícios": beneficios,
        "Status": status,
        "TRL": trl,
        "Inventores": list(inventores_set),
        "Departamento": departamento,
        "Contato": contato,
        "URL": url,
    }

In [ ]:
def escolher_formato():
    """
    Permite ao usuário a escolha do formato de exportação.
    """
    print("\n" + "="*50)
    print("Escolha o Formato de exportação: ")
    print(" [1] - JSON")
    print(" [2] - CSV")

    while True:
        escolha = input("Digite a sua escolha 1 ou 2: ").strip()
        if escolha in [ '1', '2']:
            return 'json' if escolha == '1' else 'csv'
        print("Opção inválida!, Digite 1 ou 2.")

In [ ]:
def exportar_dados(dados, formato):
    """
    Exporta os dados em JSON ou CSV
    """
    if formato == 'json':
        nome_arquivo = "tecnologias_ufc.json"
        with open(nome_arquivo, 'w', encoding='utf-8') as f:
            json.dump(dados, f, indent=2, ensure_ascii=False)
        print(f"Sucesso!, Dados exportados para: {nome_arquivo}")

    elif formato == 'csv':
        nome_arquivo = "tecnologias_ufc.csv"
        if not dados:
            print("Nenhum dado para exportar.")
            return

        colunas = [
            "Título", "Categoria", "Descrição", "Benefícios",
            "Status", "TRL", "Inventores", "Departamento", "Contato", "URL"
        ]

        with open(nome_arquivo, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=colunas)
            writer.writeheader()

            for item in dados:
                row = {}
                for coluna in colunas:
                    valor = item.get(coluna, '')
                    if isinstance(valor, list):
                        row[coluna] = '; '.join(valor)
                    else:
                        row[coluna] = valor
                writer.writerow(row)

        print(f"Sucesso!, Dados exportados para: {nome_arquivo}")

In [ ]:
def main():
    """
    Função principal que chama todas as outras para o processo de raspagem completo
    """

    print("--- Iniciando Processo de Raspagem ---")

    categorias = analisar_primeira(url_pag_inicial)
    if not categorias:
        print("Nenhuma categoria encontrada.")
        return

    lista_tecnologias = analisar_subpaginas(categorias)
    if not lista_tecnologias:
        print("Nenhuma tecnologia encontrada")
        return

    print(f"\n --- Iniciando extração de {len(lista_tecnologias)} tecnologias em paralelo ---")

    dados_finais = []

    max_workers = 6
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(extrair_info_tecnologia, tech['url']): (i, tech)
            for i, tech in enumerate(lista_tecnologias)
        }

        for future in as_completed(futures):
            i, tech = futures[future]
            info = future.result()
            if info:
                info['Categoria'] = tech.get('categoria', 'Não classificado')
                dados_finais.append(info)
                print(f"Extraído ({len(dados_finais)} /{len(lista_tecnologias)})")

    print(f"--- Extração Concluída. Total de {len(dados_finais)} itens ---")

    formato = escolher_formato()
    exportar_dados(dados_finais, formato)


if __name__ == "__main__":
    main()

--- Iniciando Processo de Raspagem ---
Buscando categorias em https://ufcinova.ufc.br/pt/vitrinetecnologica/
Categorias encontradas: 11

Processando categoria: ALIMENTOS
Acessando página: https://ufcinova.ufc.br/pt/category/vitrine-tecnologica/segmento-alimentos/
Acessando página: https://ufcinova.ufc.br/pt/category/vitrine-tecnologica/segmento-alimentos/page/2/
Acessando página: https://ufcinova.ufc.br/pt/category/vitrine-tecnologica/segmento-alimentos/page/3/
Acessando página: https://ufcinova.ufc.br/pt/category/vitrine-tecnologica/segmento-alimentos/page/4/

Processando categoria: COSMÉTICOS
Acessando página: https://ufcinova.ufc.br/pt/category/vitrine-tecnologica/segmento-cosmeticos/
Acessando página: https://ufcinova.ufc.br/pt/category/vitrine-tecnologica/segmento-cosmeticos/page/2/

Processando categoria: QUÍMICO
Acessando página: https://ufcinova.ufc.br/pt/category/vitrine-tecnologica/segmento-quimico/
Acessando página: https://ufcinova.ufc.br/pt/category/vitrine-tecnologica/seg